In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext
sc

21/09/17 11:12:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
df_read = spark.read.csv("./offensive_comment_small.csv",header = True, sep=';')

print(df_read.count())

10000


In [21]:
df_read = df_read.withColumn("toxic",df_read["toxic"].cast("int"))
df_read = df_read.withColumn("severe_toxic",df_read["severe_toxic"].cast("int"))
df_read = df_read.withColumn("obscene",df_read["obscene"].cast("int"))
df_read = df_read.withColumn("threat",df_read["threat"].cast("int"))
df_read = df_read.withColumn("insult",df_read["insult"].cast("int"))
df_read = df_read.withColumn("identity_hate",df_read["identity_hate"].cast("int"))

df_read = df_read.dropna()

In [27]:
import pyspark.ml as SparkML

# Tokenizer converts sentences to seperate words
tokenizer = SparkML.feature.Tokenizer(inputCol="comment_text",outputCol="words")

df_preparation = tokenizer.transform(df_read)

# StopWordsRemover removes useless words
wordsFilter = SparkML.feature.StopWordsRemover(inputCol="words",outputCol="words_cleaned")
df_preparation = wordsFilter.transform(df_preparation)

# preparing the model
cv = SparkML.feature.CountVectorizer(inputCol="words_cleaned",outputCol="features")
model = cv.fit(df_preparation)
pred = model.transform(df_preparation)

print(pred)



DataFrame[id: string, comment_text: string, toxic: int, severe_toxic: int, obscene: int, threat: int, insult: int, identity_hate: int, words: array<string>, words_cleaned: array<string>, features: vector]


In [ ]:
vector = SparkML.feature.VectorAssembler(inputCols=["toxic","severe_toxic","obscene","threat","insult","identity_hate"], outputCol="features")

df_transform = vector.transform(tokenizer.transform(df_read.dropna()).select("words","toxic","severe_toxic","obscene","threat","insult","identity_hate"))